# Exploration and Exploitation

In [15]:
using PGFPlots
using Interact
using Reactive
using Distributions
include("helpers.jl")
include("bandits.jl");

## Multi-Armed Bandit Problems

In [16]:
using Random
Random.seed!(2)
arms = 3
b = Bandit(arms)
banditTrial(b)

┌ Warning: attempting to remove probably stale pidfile
│   path = /home/zach/.jlassetregistry.lock
└ @ Pidfile /home/zach/.julia/packages/Pidfile/DDu3M/src/Pidfile.jl:260


(div { className="field interact-widget" }
    (button { attributes=Dict{Any, Any}("data-bind" => "click: function (){this.clicks(this.clicks()+1)}, css: {'is-loading' : loading}") className="is-medium button is-primary" style=Dict{String, Any}() }
    "Arm 1"))

Observable("0 wins out of 0 tries (NaN percent)")

(div { className="field interact-widget" }
    (button { attributes=Dict{Any, Any}("data-bind" => "click: function (){this.clicks(this.clicks()+1)}, css: {'is-loading' : loading}") className="is-medium button is-primary" style=Dict{String, Any}() }
    "Arm 2"))

Observable("0 wins out of 0 tries (NaN percent)")

(div { className="field interact-widget" }
    (button { attributes=Dict{Any, Any}("data-bind" => "click: function (){this.clicks(this.clicks()+1)}, css: {'is-loading' : loading}") className="is-medium button is-primary" style=Dict{String, Any}() }
    "Arm 3"))

Observable("0 wins out of 0 tries (NaN percent)")

(div { className="interact-widget" }
    (div { className="interact-flex-row interact-widget" }
    (div { className="interact-flex-row-left" }
      (label { className="interact " style=Dict{Any, Any}(:padding => "5px 10px 0px 10px") }
        "True Params"))
    (div { className="interact-flex-row-center" }
      (div { attributes="data-bind" => "foreach : options_js" className="buttons has-addons is-centered" }
        (span { attributes=Dict("data-bind" => "click: function (){\$root.update(val)}, css: {'is-primary is-selected' : \$root.index() == val, 'interact-widget is-medium button ' : true}") }
          (label { attributes=Dict("data-bind" => "text : key") }))))
    (div { className="interact-flex-row-right" }
      (div))))

Observable("")

## Undirected Exploration Strategies

In [3]:
struct AlwaysPull <: BanditPolicy
    arm::Int
end

arm(b::AlwaysPull, s::BanditStatistics) = b.arm;

UndefVarError: UndefVarError: `BanditPolicy` not defined

In [4]:
mutable struct ExploreThenCommit <: BanditPolicy
    k::Int
    commitment::Int
end
ExploreThenCommit(k) = ExploreThenCommit(k, 0)
function reset!(p::ExploreThenCommit)
    p.commitment = 0
    return nothing
end

function arm(b::ExploreThenCommit, s::BanditStatistics) 
    if sum(s.numTries) < b.k
        return rand(1:numArms(s))
    elseif sum(s.numTries) == b.k
        b.commitment = argmax(winProbabilities(s))
        return b.commitment
    else
        return b.commitment
    end
end;

UndefVarError: UndefVarError: `BanditPolicy` not defined

In [5]:
struct EpsGreedy <: BanditPolicy
    eps::Real 
end

function arm(b::EpsGreedy, s::BanditStatistics)
    if rand() < b.eps
        return rand(1:numArms(s))
    else
        ps = winProbabilities(s)
        return rand(findall(ps .== maximum(ps))) # like argmax but breaks ties randomly
    end
end;

UndefVarError: UndefVarError: `BanditPolicy` not defined

In [6]:
steps = 50
iterations = 10000
params = rand(Random.Xoshiro(1), 10)
display(params)
bandit = Bandit(params)
# bandit = Bandit(collect(1:-0.2:0.1))
@manipulate for epsgreedy in exp10.(-3:0.2:0), commit in 0:10:50
    policies = [
        "explore-commit" => ExploreThenCommit(commit),
        "eps greedy" => EpsGreedy(epsgreedy),
    ]
    curves = [Plots.Linear([1,steps], fill(maximum(params),2), legendentry="best arm", style="very thick, dashed", mark="none")]
    append!(curves, learningCurves(bandit, policies, steps=steps, iterations=iterations))
    Axis(curves, style="legend pos=south east", ymin=0, ymax=1, xmin=0, xmax=steps, xlabel="Pulls", ylabel="Average success", width="15cm", height="10cm")
end

10-element Vector{Float64}:
 0.0491718221481211
 0.11907881640750706
 0.3932710232252806
 0.024094310524527707
 0.6918572875342215
 0.7675180540873912
 0.08725304891274233
 0.8557176841095734
 0.8025607099234905
 0.661425351684768

UndefVarError: UndefVarError: `Bandit` not defined

## Directed (non-Bayesian) Exploration Strategies

In [7]:
struct SoftMax <: BanditPolicy
    precision::Float64 
end

function arm(b::SoftMax, s::BanditStatistics)
    p = exp.(b.precision * winProbabilities(s))
    p = p / sum(p)
    D = Categorical(p)
    return rand(D)
end;

UndefVarError: UndefVarError: `BanditPolicy` not defined

In [8]:
struct UCB <: BanditPolicy
    c::Float64
end
    
function arm(p::UCB, s::BanditStatistics)
    logn = log(max(2, sum(s.numTries)))
    ucbs = winProbabilities(s) .+ p.c*sqrt.(logn./s.numTries)
    return rand(findall(ucbs .== maximum(ucbs))) # like argmax but breaks ties randomly
end    

UndefVarError: UndefVarError: `BanditPolicy` not defined

In [9]:
steps = 100
iterations = 1000
params = rand(Random.Xoshiro(1), 10)
display(params)
bandit = Bandit(params)
# bandit = Bandit(collect(1:-0.2:0.1))
@manipulate for epsgreedy in exp10.(-3:0.2:0), softmax in 0:5:40, ucb_c in exp10.(range(-2, 0, 8))
    policies = [
        "softmax ($softmax)" => SoftMax(softmax),
        "eps greedy ($epsgreedy)" => EpsGreedy(epsgreedy),
        "ucb ($ucb_c)" => UCB(ucb_c),
    ]
    curves = [Plots.Linear([1,steps], fill(maximum(params),2), legendentry="best arm", style="very thick, dashed", mark="none")]
    append!(curves, learningCurves(bandit, policies, steps=steps, iterations=iterations))
    Axis(curves, style="legend pos=south east", ymin=0, ymax=1, xmin=0, xmax=steps, xlabel="Pulls", ylabel="Average success", width="15cm", height="10cm")
end

10-element Vector{Float64}:
 0.0491718221481211
 0.11907881640750706
 0.3932710232252806
 0.024094310524527707
 0.6918572875342215
 0.7675180540873912
 0.08725304891274233
 0.8557176841095734
 0.8025607099234905
 0.661425351684768

UndefVarError: UndefVarError: `Bandit` not defined

## Bayesian Model Estimation

In [10]:
Random.seed!(3)
arms = 2
b = Bandit(arms)
banditEstimation(b)

UndefVarError: UndefVarError: `Bandit` not defined

In [11]:
# Select arm with highest alpha upper confidence bound
struct IntervalExploration <: BanditPolicy
    alpha::Real
end

function arm(b::IntervalExploration, s::BanditStatistics)
    qs = [quantile(Beta(s.numWins[i] + 1, s.numTries[i] - s.numWins[i] + 1), b.alpha) for i in 1:length(s.numWins)]
    return rand(findall(qs .== maximum(qs))) # like argmax but breaks ties randomly
end;

UndefVarError: UndefVarError: `BanditPolicy` not defined

In [12]:
struct ThompsonSampling <: BanditPolicy end

function arm(p::ThompsonSampling, s::BanditStatistics)
    params = rand.([Beta(s.numWins[i] + 1, s.numTries[i] - s.numWins[i] + 1) for i in 1:length(s.numWins)])
    return argmax(params)
end

UndefVarError: UndefVarError: `BanditPolicy` not defined

In [13]:
steps = 100
iterations = 1000
params = rand(Random.Xoshiro(1), 10)
display(params)
bandit = Bandit(params)
# bandit = Bandit(collect(1:-0.2:0.1))
@manipulate for epsgreedy in exp10.(-3:0.2:0), interval in 0.5:0.05:1, ucb_c in exp10.(range(-2, 0, 8))
    policies = [
        # "eps greedy ($epsgreedy)" => EpsGreedy(epsgreedy),
        # "ucb ($ucb_c)" => UCB(ucb_c),
        "interval" => IntervalExploration(interval),
        "thompson sampling" => ThompsonSampling(),
    ]
    curves = [Plots.Linear([1,steps], fill(maximum(params),2), legendentry="best arm", style="very thick, dashed", mark="none")]
    append!(curves, learningCurves(bandit, policies, steps=steps, iterations=iterations))
    Axis(curves, style="legend pos=south east", ymin=0, ymax=1, xmin=0, xmax=steps, xlabel="Pulls", ylabel="Average success", width="15cm", height="10cm")
end

10-element Vector{Float64}:
 0.0491718221481211
 0.11907881640750706
 0.3932710232252806
 0.024094310524527707
 0.6918572875342215
 0.7675180540873912
 0.08725304891274233
 0.8557176841095734
 0.8025607099234905
 0.661425351684768

UndefVarError: UndefVarError: `Bandit` not defined